# 机器学习中类别数据编码

机器学习模型的输入数据一般分为数值(Numerical)型和类别(Categorical)型,数值数据规范化(Normalization)后可以直接输入模型，但类别型数据可能是代码类别的编号或者字符串，一般不能直接输入模型，需要先进行类别编码处理。

## 类别数值化

类别编码首先是将类别数值化，比如将字符串映射为0,1,2...等代表类别的数值，然后再对数值进行编码。类别数值化是要分为三种情况：
1. 纯类别。[cat, dog, mouse]，数值化为变为[1,2,3]，这些类别之间即没有顺序，也没有数值所代表的关系，比如2=2+1
2. 顺序(Ordinal)类别.[Excellent, good, average]，数值化[3,2,1]，2>1>0，但是没有数值代表的关系，比如2=2+1 
3. 数值(Cardinal)类别。[\$1, \$2, \$5]，数值化后为[1,2,5]，这种就是即有顺序也有数值关系

当纯类别数据的类别很多时，经常使用Hash给类别分配随机数值标签
$$ feature\_id = hash(raw\_feature) \% hash\_bucket\_size $$

顺序类别则需要按顺序分配数值标签，数值类别直接使用本身的数值，而且可以不需要进一步编码可以直接输入模型。对于纯类别和顺序类别数据，数值化后需要进一步编码。当在数据充足时，如何编码的影响并不大，总是是学到一个最佳的匹配模型，但实际中我们的数据总是有限的，所以要根据情况选择最适用的编码方式以提高模型的准确性。

## 基础编码方式

### One-hot Encoding

One-hot是一组N维的向量，N是类别的数量，向量只在所属类别的地方标1，其它都为0

<img src="images/one-hot.png" style="width:800px;">

In [ ]:
import pandas as pd
df = pd.DataFrame({"class" : list('abcd')})
pd.get_dummies(df)

In [ ]:
pd.get_dummies(df,drop_first=True)

### Binary Encoding

编码顺序类别时，可以使用one-hot或者直接数值化，还有一种更好的编码方式——Binary Encoding，比如0=[0,0,0,0,0,0,0,0]; 1=[0,0,0,0,0,0,0,1]; 2=[0,0,0,0,0,0,1,1]; ...; 9=[1,1,1,1,1,1,1,1]。这种编码方式会增加模型的准确度，因为它即表现出顺序关系(4 > 2)，又避免了数值关系(4 != 2*2)

In [ ]:
import numpy as np
K = 9  # number of classes
x = [1, 5, 2, 0, 3, 9]  # input
y = np.zeros((K-1, len(x)), np.int8)  # output
for i in range(len(x)):
  y[0:x[i], i] = 1
print(y)

类别很多时，One-Hot和Binary编码都会导致向量稀疏(Sparsity)的问题，导致梯度算法收敛很慢。在这种情况下，一般会直接将顺序类别当成数值类别直接使用数值化的数据输入模型，而对于纯类别数据，也最好能通过内在的关系将纯类别至少转化为顺序类别。


## 进阶编码方式-Embedding

Embedding是通过一个低阶的向量来代表类别。一般向量的维度按下面的[经验公式](https://www.tensorflow.org/guide/feature_columns?utm_medium=email&utm_source=other&utm_campaign=opencourse.discourse.deep-neural-network~opencourse.discourse.BZ5quSvBEee6gA5XksfBbg.YrF2CMk_EeiBkA6Zn5iQOg~Loawlsx8Eeis8gq8jW8C4g#indicator_and_embedding_columns)计算

$$ embedding\_dimensions =  number\_of\_categories**0.25 $$

比如有81个类别，那使用一个3阶的向量就可以了，这就大大降低了代表类别的向量维度，避免带来的稀疏问题

<img src="images/embedding.png" style="width:700px;">

这些Embedding向量值是可训练的，模型在“学习”不断适配得到最佳向量值以精确代表相应的类别，Embedding是深度学习中很重要的一个话题，一般需要大量数据训练才能达到较好的效果。Tensorflow提供了支持Embedding的[接口](https://www.tensorflow.org/guide/feature_columns)